# Social Network with Flask
Documetation
* [Flask](http://flask.pocoo.org/docs/0.10/)
* [Peewee](http://peewee.readthedocs.org/)
* [Flask-Login](http://peewee.readthedocs.org/)
* [Flask-Bcrypt](http://flask-bcrypt.readthedocs.org/)
* [Flask-WTF](https://flask-wtf.readthedocs.org/)
* [wtforms](https://wtforms.readthedocs.org/)

Notes:

`order_by` can take one or more fields to order records by by default. Each field can also have a - at the beginning to mark it as being sorted in descending order.

So why can't we do `order_by = ('joined_at', 'desc')`? Because each member of the order_by tuple is the name of a field, so that would be telling our model to order by the `'joined_at'` field and the `'desc'` field, which doesn't exist.

And since `order_by` is a tuple (you can use a list if you want), we have to include that trailing comma if there's only one tuple member.

## Code Challenge
* Import everything from the Peewee library. Create a new model named User. Give User an email attribute that is a CharField(). email should be unique.
* Now add two more attributes/fields to User. The password field should be a CharField with a max_length of 100. And the join_date field should be a DateTimeField with a default value of datetime.datetime.now.
* Finally, add a last field named bio that is a TextField. It should have an empty string for its default value. This makes it optional.

```python
import datetime

from peewee import *


class User(Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = TextField(default="")
```

## The UserMixin From Flask-Login
`pip install flask-login`

Notes

Mixins are small classes that add some specific feature. Since they're not the final class that we want to extend, they go at the beginning of our inheritance chain.

The [UserMixin](http://flask-login.readthedocs.org/en/latest/#your-user-class) docs.

## Code Challenge
* Import the UserMixin from Flask-Login. Remember that Flask extensions usually have import paths that start with flask.ext.
* Now add UserMixin to the inheritance chain of the User model.

```python
import datetime

from flask.ext.login import UserMixin
from peewee import *


class User(UserMixin, Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = TextField(default='')
```

## Cryptographic Hashing with Flask-Bcrypt
`pip install flask-bcrypt`

Notes:
* __`flask.ext.bcrypt`__ - The path where Flask Bcrypt is available.
* __`generate_password_hash()`__ - function to generate a hash from a string. Takes an optional number of rounds of hashing to use. More rounds always makes the process take longer.
* __`check_password_hash()`__ - function to check a hash against a string to see if they match.
    * `check_password_hash(previously_hashed_pwd, 'password_being_tested')`

### Playing with  Bcrypt in the Python Shell

```python
>>> from flask.ext.bcrypt import generate_password_hash
>>> help(generate_password_hash)

Help on function generate_password_hash in module flask_bcrypt:

generate_password_hash(password, rounds=None)
    This helper function wraps the eponymous method of :class:`Bcrypt`. It
    is intended to be used as a helper function at the expense of the
    configuration variable provided when passing back the app object. In other
    words this shortcut does not make use of the app object at all.

    To this this function, simple import it from the module and use it in a
    similar fashion as the method would be used. Here is a quick example::

        from flask.ext.bcrypt import generate_password_hash
        pw_hash = generate_password_hash('hunter2', 10)

    :param password: The password to be hashed.
    
>>> generate_password_hash('secret')
b'$2b$12$mnQ3M6XOMBjDNeJ38.nJW.qkSmygdxs0zvfcYcSoEwvZ.DKqWrRMi'
```
Looking at the Hashed Password:
* The `$2b` at the beginning is indicative of all Brcypt hashed passwords.
* The `$12` tells how many rounds the Brcypt function, generate_password_hash, performed. In this case, we didn't specify how many rounds we wanted, so it defaulted to 12. (DO NOT MAKE THIS NUMBER TO HIGH)
* Everything following the the third dollar sign is our hashed password
* A neat idea to use with Bcrypt:
    * Every so many times there is a failed login to the site, we increase the rounds for the hashing algorithm. This will cause the generation of a hash to take longer and longer, as well as the checking of the hash.
    
#### Checking Hashed Passwords generated by Bcrypt
```python 
>>> hashed_pw == generate_password_hash('secret')
False
```
* Merely checking the Truthiness of your 'hashed_pw' against an identical `generate_password_hash()` function will not work...
* However, when you use the `check_password_hash` method from bcrypty it does.

```python
>>> from flask.ext.bcrypt import check_password_hash
>>> check_password_hash(hashed_pw, 'secret')
True
```

## Code Challenge
* Import both generate_password_hash and check_password_hash from Flask-Bcrypt.
* Now create a function named set_password that takes a User and a string for their password. Hash the password, set the User.password attribute to the hashed password, and return the User.
* Finally write a function named validate_password that takes a user and a password. It should return True if the provided password, when hashed, matches the user's password. Otherwise, return False.

```python
from flask.ext.bcrypt import generate_password_hash, check_password_hash


def set_password(User, password):
    User.password = generate_password_hash(password)
    return User


def validate_password(User, password):
    return check_password_hash(User.password, password)
```

## Class Method
Sometimes it just doesn't make sense to make an instance of a class before we call a method on it. Python gives us a decorator named `@classmethod` that allows us to create an instance of the class from inside of it.

@classmethod

Let's talk more about @classmethod. Let's make a class to represent an email.

```python
class Email:
    to = None
    from = None
    subject = None
    content = None
```
If I want to make a new Email using the class constructor, that's easy. email = Email() and then fill in the attributes. Assuming there's a __init__() that handles setting the attributes, I can probably do that in one step.

But what if I want a method for immediately creating and sending the email? I either have to create an instance and then call .send() on the instance or I need a @classmethod way of generating one.

```python
class Email:
    to = None
    from = None
    subject = None
    content = None

    @classmethod
    def create_and_send(cls, to, from, subject, content):
        cls(to=to, from=from, subject=subject, content=content).send()
```
This won't be a benefit to every class you create, but it's often a better way of approaching use cases where you don't need the class to hang around longer than needed to perform some action.

## Code Challenge
* Add a @classmethod to User named new. It should take two arguments, email and password. The body of the method can be pass for now. Remember, @classmethods take cls as the first argument.
* Now, replace the pass in your method with a cls.create() call, using the provided email and a hash of the password.

```python
import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

database = SqliteDatabase(':memory:')

class User(Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = database
        
    @classmethod
    def new(cls, email, password):
        cls.create(
            email = email,
            password = generate_password_hash(password)
        )
```

## Before and After Requests
* __`g`__ - A global object that Flask uses for passing information between views and modules.
* __`before_request`__ - A decorator to mark a function as running before the request hits a view.
* __`after_request`__ - A decorator to mark a function as running before the response is returned.

## Code Challenge
* Import the g object from the flask library.
* Now add a function named before_request that sets g.db to the DATABASE variable in models and calls the .connect() method. The function should be decorated with the before_request decorator.
* Finally, create a function named after_request that takes a response object. The function should close the g.db connection and return the response. You should decorate the function with after_request.

```python
from flask import Flask, g

import models

app = Flask(__name__)


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()


@app.after_request
def after_request(response):
    g.db.close()
    return response
```

## LoginManager
We've used the `UserMixin` from Flask-Login. Now, let's use the `LoginManager` to set up our application so we can load users. This will get us ready to create our user CRUD.
* __`LoginManager`__ - An appliance to handle user authentication.
* __`user_loader`__ - A decorator to mark the function responsible for loading a user from whatever data source we use.

## Code Challenge
* Add a secret_key attribute to app with a random value.
* Import the LoginManager from Flask-Login.
* Now create a LoginManager instance named login_manager. Then run the init_app method, passing app as the argument.
* Finally, create a function named load_user that takes a user's id attribute as an argument. Inside the function, look up a models.User instance with the id and return it. Return None if the User doesn't exist. Decorate the function with @login_manager.user_loader.

```python
from flask import Flask, g
from flask.ext.login import LoginManager

import models

app = Flask(__name__)
app.secret_key = "asdvnsuhwerntp23qw4a0svahosrhgapsufhvasdfw4432fyr"

login_manager = LoginManager()
login_manager.init_app(app)
#login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(id):
    try:
        return models.User.get(models.User.id == id)
    except models.DoesNotExist:
        return None
```

## Flask-WTF Forms
Flask-WTF uses wtforms behind the scenes for the actual form, field, and widget creation.

`pip install flask-wtf`

Flask-WTF has C.S.R.F. protection or Cross Site Request Forgery Protection also known as one-click attack or session riding. 
* C.S.R.F. is a type of malicious exploit of a website where unauthorized commands are transmitted from a user that the web application trusts. There are many ways in which a malicious website can transmit such commands; specially-crafted image tags, hidden forms, and JavaScript XMLHttpRequests, for example, can all work without the user's interaction or even knowledge. Unlike cross-site scripting (XSS), which exploits the trust a user has for a particular site, CSRF exploits the trust that a site has in a user's browser. - Wikipedia
* The protection comes from a custom one time code that is issued with each submission of a form. If the code that comes in with the form does not match the code expected, the request is ignored.

FYI it is better not to use \w+ with Regexp because sometimes unicode doesn't play URLs (so ascii only)

## Code Challenge
* Import Form from Flask-WTF. Import StringField and PasswordField from wtforms. Lastly, import DataRequired, Email, and Length from wtforms.validators.
* Create a new Form class named SignUpForm. Give it two fields, email and password. email should be a StringField and password should be a PasswordField.
* Add DataRequired and Email to the validators for the email field.
* Finally, add DataRequired and Length to the validators for password. Set the min for Length to 8.

```python
from flask_wtf import Form
from wtforms import StringField, PasswordField
from wtforms.validators import DataRequired, Email, Length


class SignUpForm(Form):
    email = StringField(
        validators=[
            DataRequired(),
            Email(),
        ])
    password = PasswordField(
        validators=[
            DataRequired(),
            Length(min=8)
        ])
```

## Registration View
* __`form.validate_on_submit(`__) - When the form is submitted through POST, make sure the data is valid.
* Macro - A custom, executable bit of templating.
* __`form.hidden_tag()`__ - Renders hidden fields inside of a hidden `<div>`.
* [More](http://jinja.pocoo.org/docs/dev/templates/#macros) information about macros.

## Macros
{% macro %} - A function in a template to repeat code on demand. Often really useful for things like form fields.

Note:
* If you're constantly getting a locked database, change your User.create_user method to the following:

```python
@classmethod
    def create_user(cls, username, email, password, admin=False):
        try:
            with DATABASE.transaction():
                cls.create(
                    username=username,
                    email=email,
                    password=generate_password_hash(password),
                    is_admin=admin)
        except IntegrityError:
            raise ValueError("User already exists")
```

## Code Challenge
* Create a macro named hide_email. It should take a User as an argument. Print out the email attribute of the User in the following format: blah@example.com for the email test@example.com. This will require splitting the email string and using a for loop.
```python
{% macro hide_email(User) %}
      {% with letters, domain = User.email.split('@') %}
        {{ letters[0] }}{% for letter in letters[1:] %}*{% endfor %}@{{ domain }}
      {% endwith %}
{% endmacro %}
```

## Login View
__`login_user`__ - Function to log a user in and set the appropriate cookie so they'll be consideredauthenticated by Flask-Login
* Creates a session on the user's browser, gives them a cookie that references their user account and says they're logged in. 
* There are more secure ways of doing such things, Flask-login Docs are a good place to start.

## Code Challenge
* Add a new view to lunch.py. The function name should be register and the route should be "/register". It should accept both GET and POST methods. For now, have it return the string "register".
* Your register() view needs to create an instance of the SignUpForm from forms. It should also render and return the register.html template. You'll need to import render_template. In the template's context, name the SignUpForm instance as form.
* Finally, update the register() view so that the form is validated on submission. If it's valid, use the models.User.new() method to create a new User from the form data and flash the message "Thanks for registering!". You'll need to import flash().

```python 
# forms.py

from flask_wtf import Form
from wtforms import StringField, PasswordField
from wtforms.validators import DataRequired, Email, Length


class SignUpForm(Form):
    email = StringField(validators=[DataRequired(), Email()])
    password = PasswordField(validators=[DataRequired(), Length(min=8)])
    
    
    
# models.py

import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

DATABASE = SqliteDatabase(':memory:')


class User(Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = DATABASE
    
    @classmethod
    def new(cls, email, password):
        cls.create(
            email=email,
            password=generate_password_hash(password)
        )


def initialize():
    DATABASE.connect()
    DATABASE.create_tables([User], safe=True)
    DATABASE.close()



# lunch.py

from flask import Flask, g, render_template, flash
from flask.ext.login import LoginManager

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route("/register", methods=("GET", "POST"))
def register():
    form = forms.SignUpForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!")
    return render_template('register.html', form=form)      
```

## Logout View
* __`logout_user()`__ - Method to remove a user's session (delete their cookie essentially) and sign them out.
* __`@login_required`__ - Decorator to mark a view as requiring a user to be logged in before they can access the view.

## Code Challenge
* Currently, "/secret" isn't protected from anonymous users. Mark it so it requires users to be logged in. You'll need to import login_required.
* Now add a view named logout() with "/logout" as its route. This view should log the user out and then redirect to "/login". You'll need to import logout_user, redirect, and probably url_for.

```python
from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)


@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"
```

## Add Some Layout
Alert!

In this video and the others in this course, you'll see me using {{ current_user.is_authenticated() }}. At the time of filming, this was the correct way to use the is_authenticated() method on the UserMixin from flask-login. BUT, as is often the case in open source, things have changed. You'll now want to always use it as a property instead. So, use {{ current_user.is_authenticated }} instead. No parentheses!

* __`current_user`__ - Global object in templates that represents the current user.
* __`is_authenticated`__ - Property on current_user that returns whether the user is authenticated or not.
* __`get_flashed_messages(with_categories=True)`__ - Gets flashed messages with their categories.

## Code Challenge
* In the nav tag in layout.html, add three links:
A link to logout() with the text "Sign Out"
A link to 'login()' with the text "Sign In"
A link to 'register()' with the text "Sign Up"
* Now use current_user.is_authenticated() to make the logout() link only show to authenticated users and the login() and register() links show to unauthenticated users.

```python
# lunch.py

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')
    


# templates/layout.html

<!doctype html>
<html>
<head>
<title>Lunch</title>
</head>
<body>

<nav>
<!-- Add menu here -->
  {% if current_user.is_authenticated() %}
  <a href="{{ url_for('logout') }}">Sign Out</a>
  {% else %}
  <a href="{{ url_for('login') }}">Sign In</a>
  <a href="{{ url_for('register') }}">Sign Up</a>
  {% endif %}
</nav>

<div class="messages">
{% with messages = get_flashed_messages() %}
{% for message in messages %}
<div>{{ message }}</div>
{% endfor %}
{% endwith %}
</div>

{% block content %}{% endblock %}
</body>
</html>
```

## Post Model
__`ForeignKeyField`__ - A field that points to another database record.
* The __`rel_model`__ attribute is asking for a class that inherits from Model for the ForeignKey to point to.
* The __`related_name`__ attribute is asking how the class in __`rel_model`__ will refer to instances of the current class.

## Code Challenge
* Create a new Model class with the name LunchOrder. Give it a TextField attribute named order.
* Now add a DateField (not DateTimeField) to the LunchOrder model. Name it date and do not give it a default value.
* Great! Now, add a ForeignKeyField to your LunchOrder model. Name it user. user should be related to the User model and the related_name should be "orders".

```python
import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

DATABASE = SqliteDatabase(':memory:')


class User(UserMixin, Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = DATABASE
    
    @classmethod
    def new(cls, email, password):
        cls.create(
            email=email,
            password=generate_password_hash(password)
        )
        
class LunchOrder(Model):
    order = TextField()
    date = DateField()
    user = ForeignKeyField(
        rel_model=User,
        related_name="orders")


def initialize():
    DATABASE.connect()
    DATABASE.create_tables([User], safe=True)
    DATABASE.close()
```

## Post Form and View
We assigned the database and the current user, from Flask Login, to the g object, Flask's global attribute that gets passed around to all of our views automatically, so now we can use it to get the current user.

Use {{ current_user.is_authenticated }}

## Code Challenge
* In forms.py, add a new form class named LunchOrderForm. It should have two fields, order and date. The order field should be a TextAreaField and date should be a DateField. Both should have DataRequired as a validator.
* Next, create a new view in lunch.py named order_lunch. Give it a route of /order and make sure it accepts both GET and POST methods. Make it return the rendered version of the lunch.html template.
* Finally, you need to instantiate, process, and send LunchOrderForm to the template. If the form is valid after submission, create a new LunchOrder object with the data and set the user to the current user.

```python
# lunch.py

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))


@app.route('/order', methods=("GET", "POST"))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            order=form.order.data,
            date=form.date.data,
            user=g.user._get_current_object()
        )
    return render_template("lunch.html", form=form)
  

@app.route('/')
def index():
    return render_template('index.html')
    
# models.py
import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

DATABASE = SqliteDatabase(':memory:')


class User(UserMixin, Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = DATABASE
    
    @classmethod
    def new(cls, email, password):
        cls.create(
            email=email,
            password=generate_password_hash(password)
        )


class LunchOrder(Model):
    order = TextField()
    date = DateField()
    user = ForeignKeyField(User, related_name="orders")

def initialize():
    DATABASE.connect()
    DATABASE.create_tables([User], safe=True)
    DATABASE.close()
    
    
# forms.py

from flask_wtf import Form
from wtforms import StringField, PasswordField, TextAreaField, DateField
from wtforms.validators import DataRequired, Email, Length


class SignUpInForm(Form):
    email = StringField(validators=[DataRequired(), Email()])
    password = PasswordField(validators=[DataRequired(), Length(min=8)])
    
class LunchOrderForm(Form):
    order = TextAreaField(validators=[DataRequired()])
    date = DateField(validators=[DataRequired()])

```

## Code Challenge
* Add a new view at /today that shows the current user's lunch order for today. It should render the "today.html" template. Don't worry about editing the template yet. Since this is about the current user, login should be required.

```python
from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/today')
@login_required
def today():
    return render_template('today.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)
```

## Code Challenge (Stream View)
* Update templates/today.html to show today's order's date and the order attribute from the order variable. Use strftime on the date with the format %Y-%m-%d.
* Now add a `<a>` with an href attribute that points to the url for the cancel_order view. The view takes one argument, order_id, which should be the id attribute of the order.

```python 
# lunch.py

import datetime

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)


@app.route('/today')
@login_required
def today():
    order = models.LunchOrder.select().where(
        models.LunchOrder.date == datetime.date.today() &
        models.LunchOrder.user == g.user._get_current_object()
    ).get()
    return render_template('today.html', order=order)
  

@app.route('/cancel_order/<int:order_id>')
@login_required
def cancel_order(order_id):
    try:
        order = models.LunchOrder.select().where(
            id=order_id,
            user=g.user._get_current_object()
        ).get()
    except models.DoesNotExist:
        pass
    else:
        order.delete_instance()
    return redirect(url_for('index'))
```

```html
<!-- today.html -->

{% extends "layout.html" %}

{% block content %}
<h1>Your lunch for today</h1>
<h2><!-- today's date -->{{ order.date.strftime('%Y-%m-%d') }}</h2>
<p><!-- print today's lunch order -->{{ order.order }}</p>
<!-- button to the route for cancel_order with order_id=order.id -->
<a href="{{ url_for('cancel_order', order_id=order.id) }}">Cancel Order</a>
{% endblock %}
```

## Relationship Model
* `indexes` - A list of indexes to create on the model. These could be fields to index for faster searches or, as in our case, fields to make unique. [More information](http://peewee.readthedocs.org/en/latest/peewee/models.html#indexes-and-unique-constraints)
* `.join()` - A query that references another table or another query.

## Code Challenge
* Add a new model named Relationship. It should have a ForeignKeyField, related to User. The field should be named from_user with a related_name of "relationships".
* Now add a second ForeignKeyField to Relationship. This one, named to_user, should also point to User and have a related_name of "related_to".
* Finally, add a third field, created_at, that's a DateTimeField with a default of datetime.datetime.now.

```python
import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

DATABASE = SqliteDatabase(':memory:')


class User(UserMixin, Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = DATABASE
    
    @classmethod
    def new(cls, email, password):
        cls.create(
            email=email,
            password=generate_password_hash(password)
        )
class Relationship(Model):
    from_user = ForeignKeyField(
                User, related_name="relationships")
    to_user = ForeignKeyField(
                User, related_name="related_to")
    created_at = DateTimeField(default=datetime.datetime.now)

class LunchOrder(Model):
    order = TextField()
    date = DateField()
    user = ForeignKeyField(User, related_name="orders")


def initialize():
    DATABASE.connect()
    DATABASE.create_tables([User, LunchOrder], safe=True)
    DATABASE.close()
```

# Code Challenge
* Add a view named follow with a route of `"/follow/<int:user_id>"`. It should be login_required. In the view, create a new Relationship with the current user as the from_user and the user with the provided ID as the to_user. Return a redirect to index.
* Now create a view for unfollowing a user. It should be at "`/unfollow/<int:user_id>`", should require a login, and should also redirect to the index view. The view should find and delete the existing Relationship instance.

```python
import datetime

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)


@app.route('/today')
@login_required
def today():
    order = models.LunchOrder.select().where(
        models.LunchOrder.date == datetime.date.today() &
        models.LunchOrder.user == g.user._get_current_object()
    ).get()
    return render_template('today.html', order=order)
  

@app.route('/cancel_order/<int:order_id>')
@login_required
def cancel_order(order_id):
    try:
        order = models.LunchOrder.select().where(
            id=order_id,
            user=g.user._get_current_object()
        ).get()
    except models.DoesNotExist:
        pass
    else:
        order.delete_instance()
    return redirect(url_for('index'))


@app.route('/follow/<int:user_id>')
@login_required
def follow(user_id):
    to_user = models.User.get(models.User.id==user_id)
    models.Relationship.create(
        from_user=g.user._get_current_object(),
        to_user=to_user
    )
    return redirect(url_for('index'))


@app.route('/unfollow/<int:user_id>')
@login_required
def unfollow(user_id):
    to_user = models.User.get(models.User.id==user_id)
    models.Relationship.get(
        from_user=g.user._get_current_object(),
        to_user=to_user).delete_instance()
    return redirect(url_for('index'))
```

## Show follower posts
* << - Query operator that works as an equivalent to Python's in keyword.

## Code Challenge
* Update templates/profile.html so that it shows the total number of lunches ordered, followers, and people followed by a user.

```python
# lunch.py

import datetime

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)


@app.route('/today')
@login_required
def today():
    order = models.LunchOrder.select().where(
        models.LunchOrder.date == datetime.date.today() &
        models.LunchOrder.user == g.user._get_current_object()
    ).get()
    return render_template('today.html', order=order)
  

@app.route('/cancel_order/<int:order_id>')
@login_required
def cancel_order(order_id):
    try:
        order = models.LunchOrder.select().where(
            id=order_id,
            user=g.user._get_current_object()
        ).get()
    except models.DoesNotExist:
        pass
    else:
        order.delete_instance()
    return redirect(url_for('index'))


@app.route('/follow/<int:user_id>')
@login_required
def follow(user_id):
    try:
        user = models.User.get(
            models.User.id == user_id
        )
        models.Relationship.create(
            from_user=g.user._get_current_object(),
            to_user=user
        )
    except (models.DoesNotExist, models.IntegrityError):
        pass
    return redirect(url_for('index'))
  

@app.route('/unfollow/<int:user_id>')
@login_required
def unfollow(user_id):
    try:
        user = models.User.get(
            models.User.id == user_id
        )
        models.Relationship.get(
            models.Relationship.from_user==g.user._get_current_object(),
            models.Relationship.to_user==user
        ).delete_instance()
    except (models.DoesNotExist, models.IntegrityError):
        pass
    return redirect(url_for('index'))


@app.route('/profile/<int:user_id>')
def profile(user_id):
    user = models.User.select().where(
        models.User.id == user_id
    ).get()
    return render_template('profile.html', user=user)
        
```
```python
# models.py

import datetime

from flask.ext.bcrypt import generate_password_hash
from flask.ext.login import UserMixin
from peewee import *

DATABASE = SqliteDatabase(':memory:')


class User(UserMixin, Model):
    email = CharField(unique=True)
    password = CharField(max_length=100)
    join_date = DateTimeField(default=datetime.datetime.now)
    bio = CharField(default='')
    
    class Meta:
        database = DATABASE
        
    @property
    def following(self):
        return (
            User
            .select()
            .join(Relationship, on=Relationship.to_user)
            .where(Relationship.from_user == self)
        )

    @property
    def followers(self):
        return (
            User
            .select()
            .join(Relationship, on=Relationship.from_user)
            .where(Relationship.to_user == self)
        )

    
    @classmethod
    def new(cls, email, password):
        cls.create(
            email=email,
            password=generate_password_hash(password)
        )


class LunchOrder(Model):
    order = TextField()
    date = DateField()
    user = ForeignKeyField(User, related_name="orders")


class Relationship(Model):
    from_user = ForeignKeyField(User, related_name="relationships")
    to_user = ForeignKeyField(User, related_name="related_to")
    created_at = DateTimeField(default=datetime.datetime.now)

    class Meta:
        database = DATABASE
        indexes = (
            (('from_user', 'to_user'), True),
        )


def initialize():
    DATABASE.connect()
    DATABASE.create_tables([User, LunchOrder], safe=True)
    DATABASE.close()

```

```html
{% extends "layout.html" %}
{% from "macro.html" import hide_email %}

{% block content %}

<h1>{{ hide_email(user) }}</h1>

<dl>
    <dt>Lunches:</dt>
    <dd><!-- total # of lunches ordered -->{{ user.orders.count() }}</dd>
    <dt>Followers:</dt>
    <dd><!-- # of followers. user.followers is a queryset of followers -->{{ user.followers.count() }}</dd>
    <dt>Following:</dt>
    <dd><!-- # of users followed. user.following is a queryset of users followed -->{{ user.following.count() }}</dd>
</dl>

{% endblock %}
```

## Handling 404's
* __`abort()`__ - Function to immediately end a request with a specified status code.
* __`errorhandler()`__ - Decorator that marks a function as handling a certain status code.
* return __`render_template('template.html'), 404`__ - The 404 on the end specifies the status code for the response.

## Code Challenge
* Import abort from flask in lunch.py.
* Add a new function named not_found that returns the string "404". Remember, error handlers need to accept a single argument, the error. Decorate this function with @app.errorhandler(404). Add , 404 after your response, too.
* Now make not_found render the "404.html" template. Add an `<h1>` to the template with the 404 error code in it. Add a message to the template, if you want.

```python
import datetime

from flask import Flask, g, render_template, flash, redirect, url_for, abort
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)


@app.route('/today')
@login_required
def today():
    order = models.LunchOrder.select().where(
        models.LunchOrder.date == datetime.date.today() &
        models.LunchOrder.user == g.user._get_current_object()
    ).get()
    return render_template('today.html', order=order)
  

@app.route('/cancel_order/<int:order_id>')
@login_required
def cancel_order(order_id):
    try:
        order = models.LunchOrder.select().where(
            id=order_id,
            user=g.user._get_current_object()
        ).get()
    except models.DoesNotExist:
        pass
    else:
        order.delete_instance()
    return redirect(url_for('index'))


@app.route('/follow/<int:user_id>')
@login_required
def follow(user_id):
    try:
        user = models.User.get(
            models.User.id == user_id
        )
        models.Relationship.create(
            from_user=g.user._get_current_object(),
            to_user=user
        )
    except (models.DoesNotExist, models.IntegrityError):
        pass
    return redirect(url_for('index'))
  

@app.route('/unfollow/<int:user_id>')
@login_required
def unfollow(user_id):
    try:
        user = models.User.get(
            models.User.id == user_id
        )
        models.Relationship.get(
            models.Relationship.from_user==g.user._get_current_object(),
            models.Relationship.to_user==user
        ).delete_instance()
    except (models.DoesNotExist, models.IntegrityError):
        pass
    return redirect(url_for('index'))


@app.route('/profile/<int:user_id>')
def profile(user_id):
    user = models.User.select().where(
        models.User.id == user_id
    ).get()
    return render_template('profile.html', user=user)


@app.errorhandler(404)
def not_found(error):
    return render_template('404.html'), 404
        
```

```html
{% extends "layout.html" %}

{% block content %}
<h1>404</h1>

<p>Wow, sorry, that page doesn't exist.</p>

<p><a href="{{ url_for('index') }}">Try again</a></p>

{% endblock %}
```